In [20]:
import gym
import d4rl
import sys
from pathlib import Path
import numpy as np
from stable_baselines3.ppo import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize

PROJECT_ROOT_DIR = Path().absolute().parent
print(PROJECT_ROOT_DIR)

sys.path.append(str(PROJECT_ROOT_DIR))
print(sys.path)

from utils.sb3_env_utils import make_env
from utils.load_data import load_data, load_data_from_my_cache, scale_obs, split_data
from rollout.load_data import load_data as load_data_from_hd5
from utils.sb3_env_wrappers import ScaledObservationWrapper
from models.sb3_model import PPOWithBCLoss
from utils.sb3_evaluate_kl import evaluate_policy_with_kl

/home/hs/dev_ai/codes/IRPO/exp_on_d4rl
['/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages', '/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages', '/home/hs/anaconda3/envs/o2o/lib/python38.zip', '/home/hs/anaconda3/envs/o2o/lib/python3.8', '/home/hs/anaconda3/envs/o2o/lib/python3.8/lib-dynload', '', '/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages', '/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/setuptools/_vendor', '/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/pybullet_envs', '/home/hs/dev_ai/codes/IRPO/exp_on_d4rl', '/home/hs/dev_ai/codes/IRPO/exp_on_d4rl', '/home/hs/dev_ai/codes/IRPO/exp_on_d4rl', '/home/hs/dev_ai/codes/IRPO/exp_on_d4rl']


In [21]:
# ENV = "halfcheetah"
ENV = "hopper"
BC_EXPERIMENT_NAME = f"iter_3/medium_{ENV}_100epochs_loss_1_annealing"
RL_BC_EXPERIMENT_NAME = f"iter_3/medium_{ENV}_1e7steps_8envs_loss_1_annealing"
ENV_NAME = f"{ENV}-medium-v2"
GAMMA = 0.98
# EXPERT_DATA_CACHE_DIR = "rollout/data/iter_2/medium_halfcheetah_1e7steps_8envs_loss_1_annealing.hdf5"
EXPERT_DATA_CACHE_DIR = "rollout/data/iter_2/medium_hopper_1e7steps_8envs_loss_1_annealing.hdf5"
# EXPERT_DATA_CACHE_DIR = "cache"

In [22]:
origin_env = gym.make(ENV_NAME)

if EXPERT_DATA_CACHE_DIR == "cache":
    print("load data from d4rl cache.")
    obs, acts, infos = load_data(origin_env)
else:
    data_file: Path = PROJECT_ROOT_DIR / EXPERT_DATA_CACHE_DIR
    print(f"load data from {str(data_file.absolute())}")
    dataset = load_data_from_hd5(str(data_file.absolute()))
    obs, acts, infos = load_data_from_my_cache(dataset)

scaled_obs, scaler = scale_obs(obs)
env = ScaledObservationWrapper(env=origin_env, scaler=scaler)

load data from /home/hs/dev_ai/codes/IRPO/exp_on_d4rl/rollout/data/iter_2/medium_hopper_1e7steps_8envs_loss_1_annealing.hdf5


load datafile: 100%|██████████| 9/9 [00:00<00:00, 14.47it/s]


In [23]:
bc_policy_save_dir = PROJECT_ROOT_DIR / "checkpoints" / "bc" / BC_EXPERIMENT_NAME
bc_ppo = PPOWithBCLoss.load(str((bc_policy_save_dir / "bc_checkpoint").absolute()))

verbose:  0


In [24]:
rl_bc_policy_save_dir = PROJECT_ROOT_DIR / "checkpoints" / "rl" / RL_BC_EXPERIMENT_NAME
rl_bc_ppo = PPOWithBCLoss.load(str((rl_bc_policy_save_dir / "best_model").absolute()), env=env)

verbose:  0


/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [25]:
evaluate_policy_with_kl(model_teacher=bc_ppo, model_student=rl_bc_ppo, sample_model=bc_ppo, env=env, n_eval_episodes=10)

/home/hs/dev_ai/codes/IRPO/exp_on_d4rl/utils/sb3_evaluate_kl.py:84: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(3195.3837537527083,
 14.743818061427817,
 1.071204052931675e+23,
 3.132203402847052)

In [26]:
evaluate_policy_with_kl(model_teacher=rl_bc_ppo, model_student=bc_ppo, sample_model=rl_bc_ppo, env=env, n_eval_episodes=10)

/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/hs/dev_ai/codes/IRPO/exp_on_d4rl/utils/sb3_evaluate_kl.py:84: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(3257.113592207432, 5.672459996405382, inf, 115.85964490978718)

In [27]:
def get_d4rl_normalized_score(env_name, score):
    # Reference scores from D4RL paper/wiki
    REF_SCORES = {
        'halfcheetah-medium-v2': {'random': -280.18, 'expert': 12135.0},
        'hopper-medium-v2': {'random': 20.27, 'expert': 3234.3},
        'walker2d-medium-v2': {'random': 1.62, 'expert': 4592.3}
    }
    
    if env_name not in REF_SCORES:
        return None
        
    ref = REF_SCORES[env_name]
    normalized_score = (score - ref['random']) / (ref['expert'] - ref['random']) * 100
    return normalized_score

mean_reward, std_reward, mean_kl, mean_act_dist_kl = evaluate_policy_with_kl(model_teacher=bc_ppo, model_student=rl_bc_ppo, sample_model=rl_bc_ppo, env=env, n_eval_episodes=10)
normalized_score = get_d4rl_normalized_score(ENV_NAME, mean_reward)

print(f"Mean Reward: {mean_reward:.2f} +/- {std_reward:.2f}")
print(f"Normalized Score: {normalized_score:.2f}")
print(f"KL (Teacher->Student): {mean_kl}")
print(f"Action Dist KL: {mean_act_dist_kl}")

/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/hs/dev_ai/codes/IRPO/exp_on_d4rl/utils/sb3_evaluate_kl.py:84: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean Reward: 3255.12 +/- 5.59
Normalized Score: 100.65
KL (Teacher->Student): 38.181136536691234
Action Dist KL: 3.1414043052077294
